<a href="https://colab.research.google.com/gist/virattt/ba0b660cdcaf4161ca1e6e5d8b5de4f8/langgraph-financial-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook includes my code for creating a financial agent using [LangGraph](https://github.com/langchain-ai/langgraph).

The agent has two tools:

1. Extract a ticker from a user query.
2. Given a ticker, get its latest price using [Polygon](https://polygon.io/).

You will need two things to run the code:

1. OpenAI API key ([link](https://platform.openai.com/account/api-keys))
2. Polygon API key ([link](https://polygon.io/))

I've tried to make the code as easy as possible to read and run.  If you have any questions, please feel free to message me on [X](https://twitter.com/virattt)!

## Step 0 - Install dependencies

In [1]:
!pip install langgraph
!pip install -U langchain langchain_openai langchainhub


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/96/94/c31f58c7a7f470d5665935262ebd7455c7e4c7782eb525658d3dbf4b9403/urllib3-2.1.0-py3-none-any.whl.metadata
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
polygon-api-client 1.13.4 requires urllib3<2.0.0,>=1.26.9, but you have urllib3 2.1.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY', '')

## Step 1 - Define the tools that our agent will use

In [3]:
from langchain.tools import tool
from agents.plant import Plant
from langchain.schema import HumanMessage, SystemMessage

@tool
def feel(plant: Plant = None) -> str:
    """
    How your sensors are responding
    """
    plant = plant or Plant()
    return plant.feeling

## Step 2 - Create the LLM

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_community.llms import OpenLLM


tools = [feel]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = VLLM(
    model="shadowml/Daredevil-7B",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=128,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

ImportError: Could not import vllm python package. Please install it with `pip install vllm`.

## Step 3 - Define the agent and related functions

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish


# Define the agent
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    observation = tool_to_use.invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"

## Step 4 - Define and compile the agent graph

In [ ]:
from langgraph.graph import END, Graph

workflow = Graph()

workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools)

# Set the entrypoint as `agent`
workflow.set_entry_point("agent")

# Add the edges of our agent graph
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "exit": END
    }
)
workflow.add_edge('tools', 'agent')


# Compile the graph into a LangChain Runnable
chain = workflow.compile()

## Step 5 - Query our agent for the latest stock price!

In [ ]:
result = chain.invoke({"input": "What did you had for lunch today?", "intermediate_steps": []})
output = result['agent_outcome'].return_values["output"]
print(output)